## Data Analysis for EPL Predictor
We format the premier league data sourced publicly from https://www.football-data.co.uk/englandm.php by adding rankings for each team based on previous seasons 

In [1]:
import pandas as pd 
import numpy as np
import os 

## Exploratory Data Analysis 
We analyze our dataset and understand the relationships between different features.

In [2]:
df = pd.read_csv("C:\Computer Science\prempredictor\data\\2000_2001.csv")
print(df.shape)

(380, 45)


In [3]:
df.head

<bound method NDFrame.head of     Div      Date       HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
0    E0  19/08/00       Charlton       Man City     4     0   H     2     0   
1    E0  19/08/00        Chelsea       West Ham     4     2   H     1     0   
2    E0  19/08/00       Coventry  Middlesbrough     1     3   A     1     1   
3    E0  19/08/00          Derby    Southampton     2     2   D     1     2   
4    E0  19/08/00          Leeds        Everton     2     0   H     2     0   
..   ..       ...            ...            ...   ...   ...  ..   ...   ...   
375  E0  19/05/01       Man City        Chelsea     1     2   A     1     1   
376  E0  19/05/01  Middlesbrough       West Ham     2     1   H     2     1   
377  E0  19/05/01      Newcastle    Aston Villa     3     0   H     2     0   
378  E0  19/05/01    Southampton        Arsenal     3     2   H     0     1   
379  E0  19/05/01      Tottenham     Man United     3     1   H     1     1   

    HTR  ...   IWA   

In [4]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW',
       'AHW', 'HC', 'AC', 'HF', 'AF', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR',
       'HBP', 'ABP', 'GBH', 'GBD', 'GBA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD',
       'LBA', 'SBH', 'SBD', 'SBA', 'WHH', 'WHD', 'WHA'],
      dtype='object')